In [1]:
import init
from uuoskit import uuosapi
init.init()
test_account1 = init.test_account1

In [5]:
code = r'''
package main

import "chain"
import "chain/logger"

func main() {
    helloworld := []byte("hello,world")

    hash160 := chain.Sha1(helloworld);
    logger.Println(hash160[:])

    chain.AssertSha1(helloworld, hash160)

    hash256 := chain.Sha256(helloworld)
    logger.Println(hash256[:])
    chain.AssertSha256(helloworld, hash256)

    hash512 := chain.Sha512(helloworld)
    chain.Println(hash512[:])
    chain.AssertSha512(helloworld, hash512)

    hash := chain.Ripemd160(helloworld);
    chain.Println(hash[:])
    chain.AssertRipemd160(helloworld, hash);
}
'''

code = uuosapi.compile(test_account1, code, src_type=2)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [6]:
r = uuosapi.push_action(test_account1, 'sayhello', b'hello,world', {test_account1:'active'})
print(r['processed']['action_traces'][0]['console'])

74f4f4eb1947b9ca08e5e68d04d081808777f9a0 
77df263f49123356d28a4a8715d25bf5b980beeeb503cab46ea61ac9f3320eda 
2958f052052ce5c280fc1dcf97038c4f4bf36ca5bde0531567201b391d977db699c379b4d31c8b3dd75a407114104aecb84f8ca11cad67b33d865dd47a72dec3 
cdbfe28a68bcc57be34868844a2bb603ba47af6e 



In [9]:
len("\x00\x1f*\x9ef\xe8\xfb\xbf\xf1\x1c_\x88\xa5\xa5\xcb\xacz\x07\x90xk\x06\xbd?\xb2\\\xa6g\xcc,*\xe5G\x01\x04[g6o\x1e\xdefuU\xb4\xf2\xeaJ\xab\x01\x08\t\x0e={~\x81\xed\x1b\x0b\x06\xf5o\xa6\x19\xeb")

66

In [40]:
code = r'''
package main

import "encoding/hex"
import "chain"
import "chain/logger"

type mystruct struct {
    text []byte
    sign [66]byte
}

func (a *mystruct) Pack() []byte {
	enc := chain.NewEncoder(5 + len(a.text) + 66)
	enc.PackBytes(a.text)
    enc.WriteBytes(a.sign[:])
	return enc.GetBytes()
}

func (a *mystruct) Unpack(data []byte) (int, error) {
	dec := chain.NewDecoder(data)
    dec.Unpack(&a.text)
    dec.Read(a.sign[:])
    return dec.Pos(), nil
}

func main() {
    data := chain.ReadActionData()
    a := mystruct{}
    a.Unpack(data)
    logger.Println(a.text, a.sign[:])

    hash := chain.Sha256(a.text)
    pub := chain.RecoverKey(hash, a.sign[:])
    logger.Println("pub:", hex.EncodeToString(pub))
    chain.AssertRecoverKey(hash, a.sign[:], pub)
}
'''

abi = '''
{
   "version": "eosio::abi/1.0",
   "types": [],
  "structs": [{
      "name": "sayhello",
      "base": "",
      "fields": [
        {"name":"text", "type":"bytes"},
        {"name":"sign", "type":"signature"}
        ]
    }
  ],
  "actions": [{
      "name": "sayhello",
      "type": "sayhello",
      "ricardian_contract": ""
    }
  ],
  "tables": [
  ]
}
'''

code = uuosapi.compile(test_account1, code, src_type=2)
uuosapi.deploy_code(test_account1, code, vm_type=0)
print('done!')

done!


In [19]:
uuosapi.set_abi(test_account1, abi)

In [14]:
uuosapi.get_account(test_account1)

{'account_name': 'helloworld11',
 'head_block_num': 5489857,
 'head_block_time': '2021-08-17T08:54:45.000',
 'privileged': False,
 'last_code_update': '2021-08-17T08:54:30.000',
 'created': '2021-02-07T17:59:33.000',
 'core_liquid_balance': '100001004.3384 EOS',
 'ram_quota': 15730017,
 'net_weight': '83716500642',
 'cpu_weight': '334875932576',
 'net_limit': {'used': 74727,
  'available': '27480972494943',
  'max': '27480972569670',
  'last_usage_update_time': {'slot': 227501890},
  'current_used': 74714},
 'cpu_limit': {'used': 58524,
  'available': '39306727768501',
  'max': '39306727827025',
  'last_usage_update_time': {'slot': 227501890},
  'current_used': 58514},
 'ram_usage': 305602,
 'permissions': [{'perm_name': 'active',
   'parent': 'owner',
   'required_auth': {'threshold': 1,
    'keys': [{'key': 'EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV',
      'weight': 1}],
    'accounts': [{'permission': {'actor': 'helloworld11',
       'permission': 'eosio.code'},
      '

In [31]:
text = b'hello,world'
from uuoskit import wallet
import hashlib
h = hashlib.sha256()
h.update(text)
sign = wallet.sign_digest(h.digest(), 'EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV')
assert sign
#    args = {'text':text.hex(), 'sign': sign}
args = {'text':text.hex()}
args = {'sign': sign}
args = {'text':text.hex(), 'sign': sign}
r = uuosapi.push_action(test_account1, 'sayhello', args, {test_account1:'active'})
print(r['processed']['action_traces'][0]['console'])


68656c6c6f2c776f726c64 00202087d19bca7c4cf31e8d882ef71ea2ccab7f25d5c0d47801de4beb3a0e1dd9872ff7d006623c95afbacf3b2c6e3775ffc53e2659adeb3cd76326030081b2b9c5 
0002a891e0dd57132ed683bc875dacc961c6fd5dfae6800bc6181ab68bb848251e52 



In [35]:
len('0002a891e0dd57132ed683bc875dacc961c6fd5dfae6800bc6181ab68bb848251e52')/2

34.0

In [34]:
import base58

public_key = bytes.fromhex('02a891e0dd57132ed683bc875dacc961c6fd5dfae6800bc6181ab68bb848251e52 ')
print(public_key)

h = hashlib.new('ripemd160')
h.update(public_key)
digest = h.digest()
public_key += digest[:4]
b'EOS'+base58.b58encode(public_key)

b'\x02\xa8\x91\xe0\xddW\x13.\xd6\x83\xbc\x87]\xac\xc9a\xc6\xfd]\xfa\xe6\x80\x0b\xc6\x18\x1a\xb6\x8b\xb8H%\x1eR'


b'EOS6AjF6hvF7GSuSd4sCgfPKq5uWaXvGM2aQtEUCwmEHygQaqxBSV'